In [102]:
import os
import sys
import time
import random
import sklearn
import PIL.Image

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from pprint import pprint
from copy import deepcopy

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.4.0
matplotlib 3.3.3
numpy 1.19.2
pandas 1.1.5
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [2]:
# GPU configurations
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.allow_soft_placement=True
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
session = InteractiveSession(config=config)

## Load data

In [107]:
# Rename files
# Run this cell only once or when there are new data added to the folder
models_path = "D:\\datasets\\theme-classification\\models\\"
patterns_path = "D:\\datasets\\theme-classification\\patterns\\"

def rename_files(path, cwd):
    os.chdir(path)
    for old_filename in os.listdir(path):
        new_filename = deepcopy(old_filename)
        if ' ' in old_filename:
            new_filename = '_'.join(old_filename.split())
        if "条纹" in old_filename:
            new_filename = "stride".join(new_filename.split("条纹"))
        if "格子" in old_filename:
            new_filename = "grid".join(new_filename.split("格子"))
        if "几何图形" in old_filename:
            new_filename = "geometry".join(new_filename.split("几何图形"))
        if new_filename != old_filename:
            print(new_filename)
            os.rename(old_filename, new_filename)
    os.chdir(cwd)

current_path = os.getcwd()
# rename_files(models_path, current_path)
# rename_files(patterns_path, current_path)

In [73]:
old = "aaaz"
new = "aaaz"

new == old

True

In [70]:
'stride'.join("asdsad条纹asdasdsadas".split("条纹"))

'asdsadstrideasdasdsadas'

In [47]:
os.rename

<function nt.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)>

In [46]:
# "好" in "好的"
' ' in " a  a"

True

In [33]:
models_path = "D:\\datasets\\theme-classification\\models\\"
patterns_path = "D:\\datasets\\theme-classification\\patterns\\"

print(len(os.listdir(models_path)))
print(len(os.listdir(patterns_path)))

models_filelist = os.listdir(models_path)
patterns_filelist = os.listdir(patterns_path)

def fullpath(filelist, folderpath):
    return [folderpath + filename for filename in filelist]

models_filelist = fullpath(models_filelist, models_path)

models_count = len(models_filelist)
patterns_count = len(patterns_filelist)

# # Shuffle the datasets
# random.seed(1)
# random.shuffle(models_filelist)
# random.shuffle(patterns_filelist)

# # Construct dataframes from data
# models_df = pd.DataFrame(models_filelist)
# patterns_df = pd.DataFrame(patterns_filelist)

# Choose 90% as training set, 10% as validation set

6336
2007


In [39]:
models_df.columns = ["filepath"]

In [40]:
models_df

,filepath
0,1607929334de57ab8ec9979776545065005788df23.jpg
1,16021206240932833f4cf7e1f1e414b27884d8d3d3.jpg
2,1610943912838140758e4713b29514f7b8bdb2b0e4.jpg
3,LOVF-WD2488_V1.jpg
4,160983489694fe9e4a35457b4dc9b5b37b7e4140ab.jpg
...,...
6331,16073190387ba883c281f968b072dae9c8a45636bb.jpg
6332,15952103004a53be17d528cc9e78b10bf03f4eebc7.jpg
6333,VBRD-WD5_V1.jpg
6334,BGEN-WD871_V1.jpg


In [28]:
int(models_count * 0.1)

633

In [30]:
[].append

<function list.append>

In [29]:
int(patterns_count * 0.1)

200

In [34]:
models_filelist[:10]

['1607929334de57ab8ec9979776545065005788df23.jpg',
 '16021206240932833f4cf7e1f1e414b27884d8d3d3.jpg',
 '1610943912838140758e4713b29514f7b8bdb2b0e4.jpg',
 'LOVF-WD2488_V1.jpg',
 '160983489694fe9e4a35457b4dc9b5b37b7e4140ab.jpg',
 'VCHP-WD23_V1.jpg',
 '160756301600d4a49d81f7dd7cd6b16961a88525e8.jpg',
 '1608717428530b1ed23ec34302e7382ca115057e50.jpg',
 '16104309408668086d26cb8c358ef8807e698c4e07.jpg',
 'CILL-WD281_V3.jpg']

In [35]:
patterns_filelist[:10]

['185604-preview-small.jpg',
 '23_几何图形_6786.png',
 '186094-preview-small.jpg',
 '186056-preview-small.jpg',
 '23_几何图形_57.png',
 '186232-preview-small.jpg',
 '185764-preview-small.jpg',
 '23_几何图形_4931.png',
 '23_几何图形_2186.png',
 '185700-preview-small.jpg']

In [12]:
random.shuffle

<bound method Random.shuffle of <random.Random object at 0x000001FC82261EF8>>

In [ ]:
num_classes = 2

## Modeling with ResNet-50

In [ ]:
resnet50_fine_tune = keras.models.Sequential()
resnet50_fine_tune.add(keras.applications.ResNet50(include_top=False, weights='imagenet', pooling='avg'))
resnet50_fine_tune.add(keras.layers.Dense(num_classes, activation='softmax'))
resnet50_fine_tune.layers[0].trainable = False  # set ResNet (regarded as one layer)

resnet50_fine_tune.compile(loss="categorical_crossentropy",
                           optimizer="rmsprop",
                           metrics=["accuracy"])
resnet50_fine_tune.summary()